## ICU aggregate probabilities

In [ ]:
import json
from datetime import date

import matplotlib.pyplot as plt
import numpy as np
import requests
from scipy.stats import binom

data = json.dumps(
    {
        "horizon_dt": date.today().strftime("%Y-%m-%dT%H:%M:%S.%f"),
        "department": "gwb",
    }
)

In [ ]:
elective_preds = requests.post("http://uclvlddpragae08:5219/predict/", data).json()
nonelective_preds = requests.post("http://uclvlddpragae08:5230/predict/", data).json()

In [ ]:
elective = []
for bed_num in elective_preds["data"]:
    elective.append(bed_num["probability"])
print(
    "If the number of elective patients on list is " + str(len(elective) - 1),
    "the ML model will return a set of predictions from 0 beds, needed, up to that maximum, so our first array has "
    + str(len(elective))
    + " values, starting with zero",
)
poss_beds = list(range(0, len(elective)))
print(poss_beds)
fig, ax = plt.subplots(1, 1)
plt.scatter(poss_beds, elective)
plt.title("Distribution of elective probabilities")
plt.show()

In [ ]:
nonelective = []
for bed_num in nonelective_preds["data"]:
    nonelective.append(bed_num["probability"])
print(
    "The ML model will return predictions for a set of beds needed for nonelective patients that might arrive ranging from 0 to  "
    + str(len(nonelective) - 1),
    " so our second array has " + str(len(nonelective)) + " values, starting with zero",
)
poss_beds = list(range(0, len(nonelective)))
print(poss_beds)

In [ ]:
fig, ax = plt.subplots(1, 1)
plt.scatter(poss_beds, nonelective)
plt.title("Distribution of nonelective probabilities")
plt.show()

In [ ]:
num_on_unit = 18
prob_still_here = 0.8
onunit = binom.pmf(list(range(0, num_on_unit + 1)), num_on_unit, prob_still_here)
print(
    "Let's say there are "
    + str(num_on_unit)
    + " already on the unit each with a probability of "
    + str(prob_still_here)
    + " of still being here in 24 hours time, so our third array has "
    + str(len(onunit))
    + " values, starting with zero"
)
poss_beds = list(range(0, len(onunit)))
print(poss_beds)

In [ ]:
fig, ax = plt.subplots(1, 1)
plt.scatter(poss_beds, onunit)
plt.title("Distribution of probabilities for pats on unit")
plt.show()

In [ ]:
print(
    "The total possible number of admissions is the sum of the maximum possible from each of these separate distributions, which is "
    + str(num_on_unit + len(elective) - 1 + len(nonelective) - 1),
    "so the length of our aggregated predictions is one more than this number, as it is starting at zero",
)
tot_beds_array = list(range(0, len(onunit) + len(elective) - 1 + len(nonelective) - 1))
print(tot_beds_array)

In [ ]:
print(
    "To get a probability distribution over all these, we simply convolve them together like this: "
)
aggregated = np.convolve(onunit, np.convolve(elective, nonelective))

In [ ]:
fig, ax = plt.subplots(1, 1)
plt.scatter(tot_beds_array, aggregated)
plt.title("Distribution of aggregated probabilities")
plt.show()